In [0]:
%scala

// Configuración de la conexión JDBC
val jdbcUrl = "jdbc:postgresql://sample.postgres.database.azure.com:5432/postgres"
val connectionProperties = new java.util.Properties()
connectionProperties.setProperty("user", "adminuser")  // Reemplaza con tu usuario
connectionProperties.setProperty("password", "Acbp.1995")  // Reemplaza con tu contraseña
connectionProperties.setProperty("driver", "org.postgresql.Driver")  // Controlador JDBC

// Verificar la conexión listando tablas del sistema (por ejemplo, en la base de datos "postgres")
try {
  // Intentar leer algo del esquema público (vacío por defecto)
  val df = spark.read.jdbc(
    jdbcUrl,
    "trayecto",  //PONEMOS EN NOMBRE DE LA TABLA
    connectionProperties
  )

  // Mostrar resultados (debería mostrar tablas si la conexión es correcta)
  df.show()
  println("Conexión exitosa")
} catch {
  case e: Exception =>
    println("Error al conectar:", e)
}

+-----------+-----------+-------------------+-------------------+
id_pasajero|id_trayecto| inicio| fin|
+-----------+-----------+-------------------+-------------------+
 47| 283|2018-05-20 21:27:12|2018-08-31 08:27:52|
 100| 299|2019-03-09 14:11:15|2020-11-27 13:14:58|
 79| 277|2017-04-08 23:24:33|2018-10-15 16:12:30|
 47| 283|2018-05-20 21:27:12|2018-08-31 08:27:52|
 76| 365|2016-09-12 11:30:51|2020-02-02 11:21:04|
 72| 311|2018-10-01 11:21:01|2018-03-23 08:47:01|
 42| 333|2017-07-09 08:26:06|2020-03-11 13:10:35|
 52| 289|2017-03-06 01:10:45|2019-01-07 18:46:53|
 6| 288|2018-04-28 00:25:20|2016-04-16 19:11:05|
 31| 320|2020-06-18 06:44:30|2017-12-03 20:46:59|
 93| 349|2020-09-14 07:09:21|2017-08-06 23:28:43|
 61| 323|2017-03-31 06:31:29|2017-10-24 06:59:17|
 84| 357|2018-07-14 00:26:41|2019-07-11 03:22:08|
 21| 283|2016-05-11 08:56:18|2019-06-08 09:37:16|
 11| 351|2020-07-04 02:12:18|2020-12-09 00:47:52|
 11| 319|2016-05-09 22:11:01|2019-11-24 08:01:30|
 12| 338|2020-03-20 05:09:40|2019-07-02 13:02:08|
 39| 298|2019-02-11 10:38:45|2017-07-07 06:08:41|
 37| 320|2017-08-06 16:32:28|2017-03-29 15:41:16|
 39| 274|2019-03-28 00:24:27|2021-01-16 17:45:55|
+-----------+-----------+-------------------+-------------------+
only showing top 20 rows

Conexión exitosa
jdbcUrl: String = jdbc:postgresql://sample.postgres.database.azure.com:5432/postgres
connectionProperties: java.util.Properties = {user=adminuser, password=Acbp.1995, driver=org.postgresql.Driver}

In [0]:
%scala
// Crear una vista temporal de la tabla 'trayecto'
val df = spark.read.jdbc(
  jdbcUrl,
  "trayecto",  // Esquema del sistema para verificar tablas
  connectionProperties
)
df.createOrReplaceTempView("viajeScala")

df: org.apache.spark.sql.DataFrame = [id_pasajero: int, id_trayecto: int ... 2 more fields]

##  ID de trayecto y contar los ID de pasajeros que hicieron ese trayecto en el 2018

In [0]:
%sql
SELECT id_trayecto, COUNT(DISTINCT id_pasajero) AS num_personas
FROM viajeScala
WHERE YEAR(inicio) = 2018
GROUP BY id_trayecto
ORDER BY num_personas DESC;

id_trayecto,num_personas
297,3
288,2
308,2
278,1
306,1
328,1
338,1
317,1
310,1
369,1


##  Consultar el número de personas que hacen el mismo trayecto en 2017.

In [0]:
%sql
SELECT inicio, fin, COUNT(id_trayecto) AS num_trayectos
FROM viajeScala
GROUP BY inicio, fin
ORDER BY num_trayectos DESC
LIMIT 1;

inicio,fin,num_trayectos
2018-10-20T09:20:58Z,2017-10-29T22:14:57Z,2
